In [5]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

URL = "https://realpython.github.io/fake-jobs"

response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

conn = sqlite3.connect("jobs.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS jobs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    company TEXT,
    location TEXT,
    description TEXT,
    apply_link TEXT,
    UNIQUE(title, company, location)
)
""")
conn.commit()

job_list = []

jobs = soup.find_all("div", class_="card-content")

for job in jobs:
    title = job.find("h2", class_="title").text.strip()
    company = job.find("h3", class_="company").text.strip()
    location = job.find("p", class_="location").text.strip()

    description_tag = job.find("div", class_="description")
    description = description_tag.text.strip() if description_tag else "No description available"

    apply_tag = job.find("a", text="Apply")
    apply_link = apply_tag["href"] if apply_tag else "No link available"

    job_list.append((title, company, location, description, apply_link))

for job in job_list:
    cursor.execute("""
    INSERT OR IGNORE INTO jobs (title, company, location, description, apply_link)
    VALUES (?, ?, ?, ?, ?)
    """, job)

conn.commit()
conn.close()

print("Yangi ish e'lonlari yuklandi va bazaga saqlandi!")


Yangi ish e'lonlari yuklandi va bazaga saqlandi!


C:\Users\Hp\AppData\Local\Temp\ipykernel_50416\1637259182.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  apply_tag = job.find("a", text="Apply")


In [6]:
def export_to_csv(filename="jobs.csv"):
    conn = sqlite3.connect("jobs.db")
    df = pd.read_sql_query("SELECT * FROM jobs", conn)
    df.to_csv(filename, index=False, encoding="utf-8")
    conn.close()
    print(f"Ma'lumotlar {filename} fayliga eksport qilindi!")

export_to_csv()

Ma'lumotlar jobs.csv fayliga eksport qilindi!


In [7]:
def filter_jobs(location=None, company=None):
    conn = sqlite3.connect("jobs.db")
    cursor = conn.cursor()

    query = "SELECT * FROM jobs WHERE 1=1"
    params = []

    if location:
        query += " AND location LIKE ?"
        params.append(f"%{location}%")

    if company:
        query += " AND company LIKE ?"
        params.append(f"%{company}%")

    cursor.execute(query, params)
    results = cursor.fetchall()
    conn.close()

    for job in results:
        print(job)

filter_jobs(location="New York") 
filter_jobs(company="ABC Inc.")  
